In [14]:
import glob
import pandas as pd
import json
import requests

# Importing and Loading the datasets

In [17]:
# df = pd.concat([pd.read_csv(f) for f in glob.glob("./data/*.csv")], ignore_index=True)
# df.head()
data_set1 = pd.read_csv(r"C:\Users\praka\Downloads\ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")
data_set2 = pd.read_csv(r"C:\Users\praka\Downloads\ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
data_set3 = pd.read_csv(r"C:\Users\praka\Downloads\ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
data_set4 = pd.read_csv(r"C:\Users\praka\Downloads\ResaleFlatPricesBasedonApprovalDate19901999.csv")
data_set5 = pd.read_csv(r"C:\Users\praka\Downloads\ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")

In [16]:
df = pd.concat([data_set1,data_set2,data_set3,data_set4,data_set5])

In [18]:
df.shape

(911595, 11)

# Handling Null Values

In [19]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [20]:
# droping remaining_lease missing values
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

# MRT (Mass Rapid Transit System) Railway Transportation
*   Getting MRT coordinates for each city
*   so that we can calculate the distance afterwards using MRT coordinates



In [7]:
print(df['block'].unique())

['406' '108' '602' ... '378' '226E' '860B']


In [8]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [23]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]

In [24]:
import requests
import json

mrt_lat = []
mrt_long = []

for i in range(len(list_of_mrt)):
    query_address = list_of_mrt[i]
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': query_address,
        'format': 'json',
    }

    try:
        resp = requests.get(base_url, params=params)
        resp.raise_for_status()  # Raise an exception for HTTP errors
        data_mrt = resp.json()
    except requests.exceptions.RequestException as e:
        print(f"Error in HTTP request for {query_address}: {e}")
        continue
    except ValueError as e:
        print(f"Error decoding JSON for {query_address}: {e}")
        continue

    if data_mrt:
        result = data_mrt[0]
        mrt_lat.append(result.get('lat', 'NotFound'))
        mrt_long.append(result.get('lon', 'NotFound'))

        print(f"{query_address}, Lat: {result.get('lat', 'NotFound')}, Long: {result.get('lon', 'NotFound')}")
    else:
        mrt_lat.append('NotFound')
        mrt_long.append('NotFound')
        print("No Results")


Jurong East MRT Station, Lat: 1.3423978, Long: 103.732544
No Results
No Results
Choa Chu Kang MRT Station, Lat: 1.3974608, Long: 103.7474009
Yew Tee MRT Station, Lat: 1.3970146, Long: 103.7472724
Kranji MRT Station, Lat: 1.4252189, Long: 103.7619891
No Results
Woodlands MRT Station, Lat: 1.4406369, Long: 103.8009592
Admiralty MRT Station, Lat: 1.4403443, Long: 103.8008888
No Results
No Results
No Results
No Results
No Results
Ang Mo Kio MRT Station, Lat: 1.3690124, Long: 103.93809295626107
No Results
Braddell MRT Station, Lat: 1.340329, Long: 103.84677474652807
Toa Payoh MRT Station, Lat: 1.3497610500000001, Long: 103.87368414801405
No Results
Newton MRT Station, Lat: 51.410247, Long: -0.2147881
No Results
Somerset MRT Station, Lat: 51.408966, Long: -0.23054
No Results
City Hall MRT Station, Lat: 1.2932192, Long: 103.852181
Raffles Place MRT Station, Lat: 1.27660565, Long: 103.84577931609854
No Results
No Results
No Results
Tampines MRT Station, Lat: 1.3530294999999999, Long: 103.94535

In [ ]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

# Getting coordinates of each HDB(Housing and Developing Board) Resale flat
in order to conduct the distance from MRT stations as well as

*   So, that we can take out the distance of flats from MRT stations (Mass Rapid Transit System).
*   And, also from CBD (Central Business District).


In [21]:
df['address'] = df['block'] + " " + df['street_name']
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,61 years 04 months,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,250000.0,60 years 07 months,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,262000.0,62 years 05 months,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,265000.0,62 years 01 month,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,265000.0,62 years 05 months,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...
37148,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,82,297 YISHUN ST 20
37149,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,69,838 YISHUN ST 81
37150,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,74,664 YISHUN AVE 4
37151,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,70,325 YISHUN CTRL


In [10]:
address_list = df['address'].unique()

In [22]:
import requests

latitude = []
longitude = []
address = []
blk_no = []
road_name = []
postal_code = []
count = 0

for row in range(len(address_list)):
    query_address = address_list[row]
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': query_address,
        'format': 'json',
    }

    try:
        resp = requests.get(base_url, params=params)
        resp.raise_for_status()  # Raise an exception for HTTP errors

        data_geo_location = resp.json()

        if data_geo_location:
            result = data_geo_location[0]
            latitude.append(result.get('lat', ''))
            longitude.append(result.get('lon', ''))
            address.append(query_address)
            address_details = result.get('address', {})
            blk_no.append(address_details.get('house_number', ''))
            postal_code.append(address_details.get('postcode', ''))
            print(f"{query_address}, Lat: {result.get('lat', '')}, Long: {result.get('lon', '')}")
        else:
            print(f"No Results for {query_address}")

    except requests.exceptions.RequestException as e:
        print(f"Error in HTTP request for {query_address}: {e}")

    except ValueError as e:
        print(f"Error decoding JSON for {query_address}: {e}")


406 ANG MO KIO AVE 10, Lat: 1.3639027, Long: 103.8558019
108 ANG MO KIO AVE 4, Lat: 1.38391, Long: 103.837471
602 ANG MO KIO AVE 5, Lat: 1.38076, Long: 103.83542544626582
465 ANG MO KIO AVE 10, Lat: 1.3662228, Long: 103.85717530845224
601 ANG MO KIO AVE 5, Lat: 1.3809870000000002, Long: 103.83507554303799
150 ANG MO KIO AVE 5, Lat: 1.3767776999999999, Long: 103.84201139498651
447 ANG MO KIO AVE 10, Lat: 1.3674932000000002, Long: 103.85593810286824
218 ANG MO KIO AVE 1, Lat: 1.3651411, Long: 103.84172788267786
571 ANG MO KIO AVE 3, Lat: 1.37021965, Long: 103.85442413260637
534 ANG MO KIO AVE 10, Lat: 1.3741543, Long: 103.85380151282592
233 ANG MO KIO AVE 3, Lat: 1.3675791, Long: 103.8367333
235 ANG MO KIO AVE 3, Lat: 1.3688408, Long: 103.846925
219 ANG MO KIO AVE 1, Lat: 1.3659903, Long: 103.84064706999999
536 ANG MO KIO AVE 10, Lat: 1.37452405, Long: 103.8540938492528
230 ANG MO KIO AVE 3, Lat: 1.36884585, Long: 103.8379051513374
570 ANG MO KIO AVE 3, Lat: 1.3699069000000001, Long: 103

KeyboardInterrupt: 

In [ ]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

In [ ]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [ ]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

# Getting the diatnce with the help of OpenMap API

In [ ]:
# Distance to nearest MRT
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [ ]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [ ]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [ ]:
df_coordinates

In [ ]:
df_coordinates.to_csv('df_coordinates.csv',index=False)

In [ ]:
df_coordinates = pd.read_csv('df_coordinates.csv')

In [ ]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

In [ ]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2023 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

# Final Combined Data

In [ ]:
df_new.to_csv('combined.csv', index = False)

In [ ]:
d.head()

In [ ]:
d = pd.read_csv('combined.csv')